In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)
import plotly
import plotly.express as px
import numpy as np
from ipywidgets import interact

In [ ]:
import pandas as pd

In [ ]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))
df = dataset.dataframe

In [ ]:
df[(df.enemy_speed > 55) & (df.mario_speed > 65)]

In [ ]:
def plot_first_and_last_pic(index1, index2):
    dir_path = f'data/imgs_series/{index1:05d}'
    files = os.listdir(dir_path)
    img_ls = [open(f'{dir_path}/{f}', 'rb').read() for f in files]
    wi_ls = [widgets.Image(value=img, format='png') for img in img_ls]
    sidebyside = widgets.HBox([wi_ls[0], wi_ls[-1]])
    print(f'mario speed: {df[df.imgs_folder_path == dir_path].mario_speed.values[0]}'
          f', enemy speed: {df[df.imgs_folder_path == dir_path].enemy_speed.values[0]}',
          f', box x: {df[df.imgs_folder_path == dir_path].box_x.values[0]}',
          f', pipe x: {df[df.imgs_folder_path == dir_path].pipe_x.values[0]}')
    display.display(sidebyside)
    
    dir_path = f'data/imgs_series/{index2:05d}'
    files = os.listdir(dir_path)
    img_ls = [open(f'{dir_path}/{f}', 'rb').read() for f in files]
    wi_ls = [widgets.Image(value=img, format='png') for img in img_ls]
    sidebyside = widgets.HBox([wi_ls[0], wi_ls[-1]])
    print(f'mario speed: {df[df.imgs_folder_path == dir_path].mario_speed.values[0]}'
          f', enemy speed: {df[df.imgs_folder_path == dir_path].enemy_speed.values[0]}',
          f', box x: {df[df.imgs_folder_path == dir_path].box_x.values[0]}',
          f', pipe x: {df[df.imgs_folder_path == dir_path].pipe_x.values[0]}')
    display.display(sidebyside)
    
interact(plot_first_and_last_pic, index1=list(range(1, len(df)-1)), index2=list(range(1, len(df)-1)))

In [ ]:
cols = ['box_x', 'pipe_x', 'enemy_speed', 'mario_speed']
fig = go.Figure()
fig = make_subplots(rows=1, cols=4)
for i, c in enumerate(cols):
    fig.add_trace(go.Histogram(x=df[c].values), row=1, col=i+1)
    fig.update_xaxes(title_text=c, row=1, col=i+1)

# Overlay both histograms
fig.update_layout(barmode='overlay', showlegend=False, title_text="Hidden state & question distributions")
# Reduce opacity to see both histograms
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_traces(opacity=0.5)
fig.show()


In [ ]:
cols = ['answer_box', 'answer_pipe', 'answer_enemy', 'answer_mario_speed']
fig = go.Figure()
for c in cols:
    fig.add_trace(go.Histogram(x=df[c].values, name=c))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_layout(title_text="Optimal answer distributions")
fig.update_xaxes(title_text="Optimal answer value")
fig.update_yaxes(title_text="Frequency")
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
def plot_answer(answer_col, col):
    fig = px.scatter_3d(df, x=col, y="mario_speed", z=answer_col)
    fig.show()
interact(plot_answer, col=['answer_box', 'answer_pipe', 'answer_enemy', 'answer_mario_speed'],
         answer_col=['box_x', 'pipe_x', 'enemy_speed', 'mario_speed'])